In [1]:
from PIL import Image;
import numpy as np;
import operator
import matplotlib.pyplot as plt;
import random;
import math;
import statistics
from sklearn.neighbors import KDTree

In [2]:
D = (255**2)*3;

def dist(PIX_A, PIX_B):
    d = (PIX_A[0] - PIX_B[0])**2 + (PIX_A[1] - PIX_B[1])**2 + (PIX_A[2] - PIX_B[2])**2;
    return math.sqrt(d/D);

In [3]:
TC = "005"

In [4]:
source_dir = "Source_Images/source_" + TC + ".JPG"
source_image = Image.open(source_dir)
source_pix = source_image.load()

In [5]:
test_dir = "Copied_Images/copy_" + TC + ".JPG"
test_image = Image.open(test_dir)
test_pix = test_image.load()

In [6]:
source_image.size, test_image.size

((1591, 865), (384, 269))

In [7]:
spread = 3;

# matrix = np.zeros((test_image.size[0], test_image.size[1]))
start_points = []

for i in range(0, test_image.size[0], 1):
    for j in range(0, test_image.size[1], 1):
        dissimilarity = 0;
        ct = 0;
        for k in range(i-spread, i+spread+1, 1):
            for l in range(j-spread, j+spread+1, 1):
                if(k == i and j == l):
                    continue;
                if(k < 0 or k >= test_image.size[0] or l < 0 or l >= test_image.size[1]):
                    continue;
                dissimilarity += dist(test_pix[k, l], test_pix[i, j]);
        
#         Diss = dissimilarity/ct;
#         matrix[i][j] = dissimilarity * statistics.variance(test_pix[i, j]);
        Diss = dissimilarity * statistics.variance(test_pix[i, j]);
        Diss = round(Diss, 2)
        start_points.append([Diss, i, j]);

start_points = sorted(start_points, key = operator.itemgetter(0));
start_points.reverse()

In [8]:
spread = 2;

# matrix = np.zeros((test_image.size[0], test_image.size[1]))
source_points = []

for i in range(0, source_image.size[0], 1):
    for j in range(0, source_image.size[1], 1):
        dissimilarity = 0;
        ct = 0;
        for k in range(i-spread, i+spread+1, 1):
            for l in range(j-spread, j+spread+1, 1):
                if(k == i and j == l):
                    continue;
                if(k < 0 or k >= source_image.size[0] or l < 0 or l >= source_image.size[1]):
                    continue;
                dissimilarity += dist(source_pix[k, l], source_pix[i, j]);
        
#         Diss = dissimilarity/ct;
#         matrix[i][j] = dissimilarity * statistics.variance(test_pix[i, j]);
        Diss = dissimilarity * statistics.variance(source_pix[i, j]);
        Diss = round(Diss, 2)
        source_points.append([Diss, i, j]);

source_points = sorted(source_points, key = operator.itemgetter(0));
source_points.reverse()

In [9]:
start_points[0], source_points[0]

([108003.72, 244, 152], [98207.74, 127, 16])

In [10]:
len(start_points), len(source_points)

(103296, 1376215)

In [11]:
class Path:
    def __init__(self):
        self.points = [];
        self.score = [];
        self.agg_score = -1;

In [12]:
def aggregate_score(scores):
    random.shuffle(scores);
    total_buckets = 100;
    max_bucket_size = len(scores)/total_buckets;
    buckets = [];
    curr_size = 0;
    curr_sum = 0;
    for sc in scores:
        curr_size += 1;
        curr_sum += sc;
        if(curr_size == max_bucket_size):
            buckets.append(curr_sum/curr_size);
            curr_sum = 0;
            curr_size = 0;
    if(curr_size > 0):
        buckets.append(curr_sum/curr_size);
    
    return statistics.median(buckets);

In [13]:
ans = Path();
ans.agg_score = -1;

In [14]:
MAX_STEP = 10;
MAX_STEP = math.floor(min(test_image.size[0], test_image.size[1])/10);

In [16]:
MAX_STEP

26

In [17]:
source_points_rgb = [];
for i in range(0, len(source_points), 1):
    X = source_points[i][1];
    Y = source_points[i][2];
    source_points_rgb.append(list(source_pix[X, Y]));

In [18]:
source_points_rgb[15] ,len(source_points_rgb), len(source_points)

([186, 89, 46], 1376215, 1376215)

In [19]:
source_points[0]

[98207.74, 127, 16]

In [20]:
pts_under_consideration = math.floor(len(source_points)*30/100)
source_points_rgb = source_points_rgb[:pts_under_consideration]
source_points = source_points[:pts_under_consideration]

In [21]:
len(source_points_rgb), len(source_points)

(412864, 412864)

In [22]:
tree = KDTree(source_points_rgb, leaf_size = 3, metric = 'euclidean')

In [23]:
di, ind = tree.query([[137.653, 86.462, 89.342]], k = 5)
ind.sort();
di.sort();

ind, di

(array([[2647, 2740, 3226, 3397, 4332]], dtype=int64),
 array([[6.30926438, 6.9514615 , 6.99119568, 8.74636021, 9.9132647 ]]))

In [32]:
RANDOM_WALK_LEN = 500;

In [33]:
ITR_LIMIT = 10;

for START in range(0, ITR_LIMIT, 1):
    print("ITR: ", START+1);
    
    start_X = start_points[START][1];
    start_Y = start_points[START][2];
#     print(start_X, start_Y)
    
    v = []
#     for i in range(0, source_image.size[0], 1):
#         for j in range(0, source_image.size[1], 1):
#             d = dist(test_pix[start_X, start_Y], source_pix[i,j]);
#             v.append([d, i, j]);
    
    S_RGB = list(source_pix[start_X, start_Y]);
    
    ds, ind = tree.query([S_RGB], k = 10000);
    for i in range(0, len(ind[0]), 1):
        X = source_points[ind[0][i]][1];
        Y = source_points[ind[0][i]][2];
        d = dist(test_pix[start_X, start_Y], source_pix[X,Y])
        v.append([d, X, Y]);
    
    v = sorted(v, key = operator.itemgetter(0));
    
    paths = [];
    for i in range(0, len(v), 1):
        p = Path();
        p.points.append([v[i][1], v[i][2]]);
        p.score.append(v[i][0]);
        paths.append(p);
    
    curr_X = start_X;
    curr_Y = start_Y;

    for itr in range(0, RANDOM_WALK_LEN, 1):
        step_X = random.randint(1, MAX_STEP);
        step_Y = random.randint(1, MAX_STEP);
        
        off_X = random.randint(0, 1);
        off_Y = random.randint(0, 1);

        if(off_X == 0): off_X = -1;
        if(off_Y == 0): off_Y = -1;

        curr_X = curr_X + off_X * step_X;
        curr_Y = curr_Y + off_Y * step_Y;

        if(curr_X < 0 or curr_X >= test_image.size[0] or curr_Y < 0 or curr_Y >= test_image.size[1]):
            curr_X = curr_X - off_X * step_X;
            curr_Y = curr_Y - off_Y * step_Y;
            continue;

        img_RGB = test_pix[curr_X, curr_Y];

        for i in range(0, len(paths), 1):
            if(paths[i].score[-1] == -1):continue;
            last_X = paths[i].points[-1][0];
            last_Y = paths[i].points[-1][1];

            new_X = last_X + off_X * step_X;
            new_Y = last_Y + off_Y * step_Y;

            if(new_X < 0 or new_X >= source_image.size[0] or new_Y == 0 or new_Y >= source_image.size[1]):
                paths[i].score.append(-1);
                continue;

            d = dist(img_RGB, source_pix[new_X, new_Y]);

            paths[i].score.append(d);
            paths[i].points.append([new_X, new_Y]);
        
    final_P = Path();
    for p in paths:
        if(p.score[-1] == -1):
            continue;

        p.agg_score = aggregate_score(p.score);
        if(p.agg_score < final_P.agg_score or final_P.agg_score == -1):
            final_P = p;

    if(final_P.agg_score == -1):
        continue;

    if(final_P.agg_score < ans.agg_score or ans.agg_score == -1):
        ans = final_P;

ITR:  1
ITR:  2
ITR:  3
ITR:  4
ITR:  5
ITR:  6
ITR:  7
ITR:  8
ITR:  9
ITR:  10


In [34]:
ans.agg_score

0.035439809353896304

In [35]:
1 - ans.agg_score

0.9645601906461037

In [36]:
output_dir = "Results/result_" + TC + ".jpg";
for i in range(0, len(ans.points), 1):
#     print(final_P.points[i]);
    source_pix[ans.points[i][0], ans.points[i][1]] = (0, 0, 0)

source_image.save(output_dir)